In [239]:
# Data-driven Newsvendor with Pricing
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline

In [240]:
#Initalization
a = 0
b = 0
p = []
D = []
u = []

optimalPrice = []
optimalQty = []
objValue = []

# BEUTEL (2012)

### Method to generate sampling of demand and price:
The function of demand in terms of price used here is D(p) = a - b\*p + u.
We assume the true demand function by randomly choose a, b, p and D. In more detail:
- market size a ~ U(1500, 2000)
- slope b ~ U(500, 1000)
- uniformly chosen sample of price p (with size = sampleSize) in range [1, 1.5] 
- D equals to the sum of prediction value plus estimation error u ~ N(0, sigma). Sigma is chosen so that the coefficient of variation at mean price p equals coefVar. 


In [241]:
# Demand - Price generation
def samplingDemand(sampleSize, coefVar):
    n = sampleSize
    cv = coefVar
    
    global a, b, p, D, u
    a = 0; b = 0; p = []; D = []; u = [] ;

    a = np.random.uniform(low = 1500, high = 2000)
    b = np.random.uniform(low = 500, high = 1000)
    p = np.random.uniform(size = [n,1])*0.5 + 1 # uniformly chosen in price range [1, 1.5]

    meanP = np.mean(p)
    sigma = (a - b*meanP)*cv
    D = np.repeat(-1, repeats = n)
    for i in range(n):
        while D[i] < 0:
            D[i] = np.random.normal(loc = a - b * p[i], scale = sigma)
    u = D - (a-b*p).reshape(n,)


#     plt.plot(p,D, 'o', label = 'D(p) = ' + str(round(a,0)) + ' - ' + str(round(b,2)) + 'p + u')
#     plt.xlabel('Price')
#     plt.ylabel('Demand')
#     plt.legend()
#     plt.title('Demand sampling')
    


### Optimization model to find the ordering quantity as a function of price
Here we use the approach presented in Beutel (2012) that integrates demand estimation and ordering optimization in a single model (instead of sequential steps) to learn about the relationship between ordering quantity and price. The main idea is to build the safety inventory (or ordering quantity in newsvendor problem) by utilizing the relationship between demand and price. The advantage of this approach is that it takes the overage and underage costs into account during estimating (i.e. different weights applied to the positive/negative errors in regression).  
  
There is a slight difference of two models below to the base models in Beutel (2012). In her models, Beutel assumes fixed holding cost and penalty cost, representing the overage and underage cost respectively. In other words, these costs are **independent with selling price**. However, here we will define the overage and underage cost as in the traditional newsvendor model, i.e.:     
- overage cost (co) = procurement cost (c)
- underage cost (cu) = selling price - procurement cost (p-c)

Therefore, the underage cost (and so the critical fractile) **depends on the selling price**. This modification is more logical especially when considering the context of price-setting newsvendor because it causes the trade-off characteristic in price decision. For example, increasing the price affects the expected profit both positively (by increasing revenue per unit) and negatively (by decreasing the demand).

Then, the objective function used here is maximization of profit, instead of minimization of inventory holding cost and shortage penalties.

#### The cost model

In [242]:
# Cost model
def costModel(sampleSize, procurementCost):
    n = sampleSize
    c = procurementCost
    
    baseCostModel = gp.Model("Base Cost model")

    #### Variables
    betaC0 = baseCostModel.addVar(lb = - GRB.INFINITY, vtype = GRB.CONTINUOUS) # ~ a
    betaC1 = baseCostModel.addVar(lb = - GRB.INFINITY, vtype = GRB.CONTINUOUS) # ~ b

    inventory = {} # = q - D (overage)
    shortage = {} # = D - q (underage)
    sales = {} # = min(D, q)
    for i in range(n):
        #overage cost per unit = c(co)
        inventory[i] = baseCostModel.addVar(lb = 0, vtype = GRB.CONTINUOUS, obj = c) 
        #underage cost per unit = p-c (cu)
        shortage[i] = baseCostModel.addVar(lb = 0, vtype = GRB.CONTINUOUS, obj = p[i] - c)

        sales[i] = baseCostModel.addVar(lb = 0, vtype = GRB.CONTINUOUS)

    #### Constraints
    for i in range(n):
        baseCostModel.addConstr(shortage[i] == D[i] - sales[i])
        baseCostModel.addConstr(inventory[i] >= betaC0 + betaC1 * p[i] - D[i])
        baseCostModel.addConstr(sales[i] <= D[i])
        baseCostModel.addConstr(sales[i] <= betaC0 + betaC1 * p[i])

    baseCostModel.optimize()
    
    return [baseCostModel.objVal, betaC0.x, betaC1.x]

In [243]:
# print("Objective: "+str(baseCostModel.objVal))
# print('Estimation of a:', betaC0.x)
# print('Estimation of b:', betaC1.x)

# print('\nOrder quantity - price function: D = ', round(betaC0.x,0),'-', abs(round(betaC1.x,2)),'* p')


# # OLS Linear regression for comparison
# reg = linear_model.LinearRegression().fit(p, D)
# coef2 = reg.coef_
# coef1 = reg.intercept_

# #Plot
# plt.plot(p,D,'o')
# plt.plot(p, betaC0.x + betaC1.x*p, label='Order quantity function line')
# plt.plot(p, coef1 + coef2*p, label = 'Demand prediction with OLS Linear regression')
# plt.legend(loc = 8)
# plt.xlabel('Price')
# plt.ylabel('Demand')
# plt.title('Cost model')
# plt.show()

#### The service level model

In [244]:
# Service level model
def SLModel(sampleSize, procurementCost, nonStockOut):
    n = sampleSize
    c = procurementCost
    
    baseSLModel = gp.Model("Base Service level model")

    #### Variables
    betaSL0 = baseSLModel.addVar(lb = - GRB.INFINITY, vtype = GRB.CONTINUOUS) # ~ a
    betaSL1 = baseSLModel.addVar(lb = - GRB.INFINITY, vtype = GRB.CONTINUOUS) # ~ b

    inventory = {}
    sales = {}
    unfulfilled = {}
    for i in range(n):
        inventory[i] = baseSLModel.addVar(lb = 0, vtype = GRB.CONTINUOUS, obj = c)   
        sales[i] = baseSLModel.addVar(lb = 0, vtype = GRB.CONTINUOUS)
        unfulfilled[i] = baseSLModel.addVar(vtype = GRB.BINARY)

    #### Constraints
    for i in range(n):
        baseSLModel.addConstr(inventory[i] >= betaSL0 + betaSL1 * p[i] - D[i])
        baseSLModel.addConstr(sales[i] <= D[i])
        baseSLModel.addConstr(sales[i] <= betaSL0 + betaSL1 * p[i])

        baseSLModel.addConstr(D[i] - unfulfilled[i]*np.max(D) <= betaSL0 + betaSL1 * p[i])

    #non-stockout (alpha level)    
    baseSLModel.addConstr(sum(unfulfilled[i] for i in range(n)) <= n*(1-nonStockOut)) 

    #fillrate (beta level)
    #baseSLModel.addConstr( sum(sales[i] for i in range(n)) >= fillRate * sum(D[i] for i in range(n)) )

    baseSLModel.optimize()
    
    return [baseSLModel.objVal, betaSL0.x, betaSL1.x]

**Results:**

In [245]:
# print("Objective: "+str(baseSLModel.objVal))
# print('Estimation of a:', betaSL0.x)
# print('Estimation of b:', betaSL1.x)

# print('\nOrder quantity - price function: D = ', round(betaSL0.x,0),'-', abs(round(betaSL1.x,2)),'* p')


# # OLS Linear regression for comparison
# reg = linear_model.LinearRegression().fit(p, D)
# coef2 = reg.coef_
# coef1 = reg.intercept_

# #Plot
# plt.plot(p,D,'o')
# plt.plot(p, betaSL0.x + betaSL1.x*p, label='Order quantity function line')
# plt.plot(p, coef1 + coef2*p, label = 'Demand prediction with OLS Linear regression')
# plt.legend(loc = 8)
# plt.xlabel('Price')
# plt.ylabel('Demand')
# plt.title('Service Level model (alpha level)')
# plt.show()

# PRICE SETTING

In [246]:
#Newsvendor with Price-setting
def NVPrice(sampleSize, procurementCost, beta0, beta1):
    n = sampleSize
    c = procurementCost
    M = 5000 #big M
    
    pricingNV = gp.Model("Pricing NV model")

    #### Variables
    price = pricingNV.addVar(lb = 1, ub = 1.5, vtype = GRB.CONTINUOUS) #same price range to avoid extrapolation
    quantity = pricingNV.addVar()

    sales = {}
    #demand = {}
    z = {} #binary variable = 1 if D = a - bp + u > 0, =0 otherwise ==> to ensure demand not negative: D = max(0, a-bp+u)

    for t in range(n):
        sales[t] = pricingNV.addVar()
        #demand[t] = pricingNV.addVar()
        z[t] = pricingNV.addVar(vtype = GRB.BINARY)

    for t in range(n):
        #pricingNV.addConstr(demand[t] >= beta0 + beta1*price + u[t]) #demand[t] >= max(0, a-bp+u)
        #pricingNV.addConstr(demand[t] <= beta0 + beta1*price + u[t] + (1-z[t])*M ) #if z=1 or a-bp+u >= 0, constraint is bounded
        pricingNV.addConstr(M*z[t] >= beta0 + beta1*price + u[t] ) #if a-bp+u > 0, z must equal to 1
        pricingNV.addConstr(sales[t] <= 0 + M*z[t] )
        pricingNV.addConstr(sales[t] <= beta0 + beta1*price + u[t] + M*(1-z[t]) )
        pricingNV.addConstr(sales[t] <= quantity)

    pricingNV.setObjective( price * sum(sales[t] for t in range(n)) / n - procurementCost*quantity) #maximize expected profit
    pricingNV.modelSense = 0 #maximization

    pricingNV.Params.NonConvex = 2 #enable Quadratic Optimization mode of Gurobi
    pricingNV.optimize()

    print("Objective: "+str(pricingNV.objVal))
    print('Optimal price:',price.x)
    print('Optima ordering quantity:', quantity.x)
    
    return [pricingNV.objVal, price.x, quantity.x]

In [253]:
def DDNV_Price(sampleSize, coefVar, procurementCost, replications, estimationModel = 'cost', nonStockOut = None):
    #Initalization
    global optimalPrice, optimalQty, objValue
    optimalPrice = []; optimalQty = []; objValue = [];
    
    for i in range(replications):
        samplingDemand(sampleSize, coefVar)
        if estimationModel == 'service' and nonStockOut is not None:
            (estObjVal, estBeta0, estBeta1) = SLModel(sampleSize, procurementCost)
        else:                
            (estObjVal, estBeta0, estBeta1) = costModel(sampleSize, procurementCost)
        print('\n',estObjVal, estBeta0, estBeta1)
        (optVal, optP, optQ) = NVPrice(sampleSize, procurementCost, estBeta0, estBeta1)
        print('\n',optVal, optP, optQ)
        
        optimalPrice.append(optP)
        optimalQty.append(optQ)
        objValue.append(optVal)
        
    return np.mean(optVal), np.mean(optimalPrice), np.mean(optimalQty)


In [248]:
DDNV_Price(sampleSize=200, coefVar = 0.3, procurementCost=0.75, replications= 100, estimationModel = 'cost', nonStockOut = None)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 602 columns and 1800 nonzeros
Model fingerprint: 0x79fd6d84
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 1e+03]
Presolve removed 400 rows and 200 columns
Presolve time: 0.01s
Presolved: 400 rows, 402 columns, 1200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     219    2.5308482e+04   0.000000e+00   0.000000e+00      0s

Solved in 219 iterations and 0.02 seconds
Optimal objective  2.530848173e+04

 25308.481733513006 1547.9339747008519 -600.4931710895646
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 402 columns and 1800 nonzeros
Model fingerprint: 0xfa50abbd
Model has 200 quadrat


Optimal solution found (tolerance 1.00e-04)
Best objective 3.274630586426e+02, best bound 3.274871819295e+02, gap 0.0074%
Objective: 327.4630586425597
Optimal price: 1.499774915265153
Optima ordering quantity: 655.1551599763753

 327.4630586425597 1.499774915265153 655.1551599763753
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 602 columns and 1800 nonzeros
Model fingerprint: 0x3a77faf4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 400 rows and 200 columns
Presolve time: 0.00s
Presolved: 400 rows, 402 columns, 1200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     211    2.5476391e+04   0.000000e+00   0.000000e+00      0s

Solved in 211 iterations and 0.02 seconds
Optimal objective  2.547639062e+04

 25476.390621244147 1296.

  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+03, 7e+03]
Found heuristic solution: objective -1.50000e+09
Presolve removed 396 rows and 198 columns
Presolve time: 0.00s
Presolved: 805 rows, 405 columns, 2011 nonzeros
Presolved model has 200 bilinear constraint(s)
Found heuristic solution: objective -0.0000000
Variable types: 403 continuous, 2 integer (2 binary)

Root relaxation: objective 3.063846e+02, 682 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  306.38460    0  202   -0.00000  306.38460      -     -    0s
H    0     0                     222.4170365  306.38460  37.8%     -    0s
H    0     0                     222.4293051  306.38460  37.7%     -    0s
     0     0  276.34550    0  201  222.42931  276.34550  24.2%     -    0s
     0     0  275.28565    0  201  222.42931  275.28565  23.8%     -    0s
     0     0  274.93485    

     0     0  283.87962    0  200  244.09016  283.87962  16.3%     -    0s
     0     0  283.87958    0  200  244.09016  283.87958  16.3%     -    0s
     0     0  283.61909    0  200  244.09016  283.61909  16.2%     -    0s
     0     0  283.35939    0  200  244.09016  283.35939  16.1%     -    0s
     0     0  283.10065    0  200  244.09016  283.10065  16.0%     -    0s
     0     0  283.10023    0  200  244.09016  283.10023  16.0%     -    0s
     0     0  282.86640    0  200  244.09016  282.86640  15.9%     -    0s
     0     0  282.86612    0  200  244.09016  282.86612  15.9%     -    0s
     0     0  282.60635    0  200  244.09016  282.60635  15.8%     -    0s
     0     0  282.34670    0  199  244.09016  282.34670  15.7%     -    0s
     0     0  282.34669    0  199  244.09016  282.34669  15.7%     -    0s
     0     0  282.08727    0  200  244.09016  282.08727  15.6%     -    0s
     0     0  281.82886    0  200  244.09016  281.82886  15.5%     -    0s
     0     0  281.82849  

 428.2455118621165 1.499999999607481 824.884735923873
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 602 columns and 1800 nonzeros
Model fingerprint: 0x5dad3ac9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 9e+02]
Presolve removed 400 rows and 200 columns
Presolve time: 0.00s
Presolved: 400 rows, 402 columns, 1200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     218    1.0236256e+04   0.000000e+00   0.000000e+00      0s

Solved in 218 iterations and 0.01 seconds
Optimal objective  1.023625644e+04

 10236.256436222879 1290.2105954451843 -743.4049971964613
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 402 columns and 1800 nonzer

...

Presolve time: 0.01s
Presolved: 400 rows, 402 columns, 1200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     225    2.8943202e+04   0.000000e+00   0.000000e+00      0s

Solved in 225 iterations and 0.02 seconds
Optimal objective  2.894320167e+04

 28943.201670608873 1534.438977470309 -522.4059556042844
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 402 columns and 1800 nonzeros
Model fingerprint: 0xaa9cf10b
Model has 200 quadratic objective terms
Variable types: 202 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [8e-01, 8e-01]
  QObjective range [1e-02, 1e-02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [7e+02, 7e+03]
Found heuristic solution: objective -1.50000e+09
Presolve removed 398 rows and 1


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  416.89954    0  202   -0.00000  416.89954      -     -    0s
H    0     0                     317.7780103  416.89954  31.2%     -    0s
     0     0  382.09854    0  201  317.77801  382.09854  20.2%     -    0s
     0     0  381.91627    0  202  317.77801  381.91627  20.2%     -    0s
     0     0  380.83947    0  200  317.77801  380.83947  19.8%     -    0s
     0     0  380.80830    0  200  317.77801  380.80830  19.8%     -    0s
     0     0  380.36106    0  135  317.77801  380.36106  19.7%     -    0s
     0     0  380.02933    0  200  317.77801  380.02933  19.6%     -    0s
     0     0  380.01980    0  200  317.77801  380.01980  19.6%     -    0s
     0     0  379.67535    0  200  317.77801  379.67535  19.5%     -    0s
     0     0  379.67287    0  200  317.77801  379.67287  19.5%     -    0s
     0     0  379.32229

     225    2.5909042e+04   0.000000e+00   0.000000e+00      0s

Solved in 225 iterations and 0.02 seconds
Optimal objective  2.590904224e+04

 25909.042237144986 1450.9769205148752 -579.6271416808711
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 402 columns and 1800 nonzeros
Model fingerprint: 0xf8e14259
Model has 200 quadratic objective terms
Variable types: 202 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [8e-01, 8e-01]
  QObjective range [1e-02, 1e-02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [7e+02, 7e+03]
Found heuristic solution: objective -1.50000e+09
Presolve removed 392 rows and 196 columns
Presolve time: 0.00s
Presolved: 809 rows, 407 columns, 2021 nonzeros
Presolved model has 200 bilinear constraint(s)
Found heuristic solution: objective -0.0000000
Variable types: 403 con

       0      handle free variables                          0s
     215    3.0302492e+04   0.000000e+00   0.000000e+00      0s

Solved in 215 iterations and 0.02 seconds
Optimal objective  3.030249153e+04

 30302.491533150896 1139.100264656271 -220.97730541885997
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 402 columns and 1800 nonzeros
Model fingerprint: 0x26813c4b
Model has 200 quadratic objective terms
Variable types: 202 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [8e-01, 8e-01]
  QObjective range [1e-02, 1e-02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [3e+02, 7e+03]
Found heuristic solution: objective -1.50000e+09
Presolve removed 398 rows and 199 columns
Presolve time: 0.00s
Presolved: 803 rows, 404 columns, 2006 nonzeros
Presolved model has 200 bilinear constraint(s)
Found 

* 2180   231             267     140.4487158  140.64213  0.14%   4.7    1s
* 3232   287             425     140.4487355  140.51736  0.05%   4.3    1s
* 3905   462             387     140.4488106  140.51736  0.05%   4.1    1s
* 4186   460             391     140.4488230  140.51736  0.05%   4.1    1s
* 5535   642             411     140.4489263  140.49322  0.03%   3.9    2s
H 6195   770                     140.4499857  140.47664  0.02%   4.0    2s
*12325  1722             540     140.4499894  140.46621  0.01%   3.5    3s
*12636  1727             551     140.4499906  140.46621  0.01%   3.5    3s
*13400  1757             545     140.4499926  140.45597  0.00%   3.5    3s

Explored 13680 nodes (48981 simplex iterations) in 3.66 seconds
Thread count was 8 (of 8 available processors)

Solution count 10: 140.45 140.45 140.45 ... 140.447

Optimal solution found (tolerance 1.00e-04)
Best objective 1.404499925439e+02, best bound 1.404559701247e+02, gap 0.0043%
Objective: 140.44999254394543
Optimal

In [258]:
import pandas as pd

results = pd.DataFrame(index = range(10), columns = ['Procurement cost', 'Optimal price', 'Optimal order QTY'])

for i in range(2):
    c = 0.1*(i+1)
    value,meanP,meanQ = DDNV_Price(sampleSize=200, coefVar = 0.3, procurementCost=c, replications= 1, estimationModel = 'cost', nonStockOut = None)
    results.iloc[i,:] = [c, meanP, meanQ] 

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 602 columns and 1800 nonzeros
Model fingerprint: 0x8e7c2a15
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+03]
Presolve removed 400 rows and 200 columns
Presolve time: 0.01s
Presolved: 400 rows, 402 columns, 1200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     214    8.2927866e+03   0.000000e+00   0.000000e+00      0s

Solved in 214 iterations and 0.01 seconds
Optimal objective  8.292786580e+03

 8292.786579628924 1948.3984866190322 -676.4952479935266
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 800 rows, 402 columns and 1800 nonzeros
Model fingerprint: 0x17b02708
Model has 200 quadrati

In [259]:
results

,Procurement cost,Optimal price,Optimal order QTY
0,0.1,1.47329,1263.91
1,0.2,1.4999,995.464
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [238]:
# Parameters

procurementCost = 0.25 
#price in : p > c [1,1.5]

nonStockOut = 0.10
fillRate = 0.9

Help on class range in module builtins:

class range(object)
 |  range(stop) -> range object
 |  range(start, stop[, step]) -> range object
 |  
 |  Return an object that produces a sequence of integers from start (inclusive)
 |  to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
 |  start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
 |  These are exactly the valid indices for a list of 4 elements.
 |  When step is given, it specifies the increment (or decrement).
 |  
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |